# Top-level imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

The tensorboard module is not an IPython extension.


#### import packages - setup

In [2]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import stats
from sklearn import metrics
import datetime
import tensorflow as tf

/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/tensorflow/pyth

In [3]:
plt.style.use('fivethirtyeight')

In [4]:
data_dir = '../data/'

In [5]:
data_files = os.listdir(data_dir)
data_files.sort()
data_files

['BroughtonSeaLice_fishData.csv',
 'BroughtonSeaLice_fishInfo.csv',
 'BroughtonSeaLice_siteData.csv',
 'BroughtonSeaLice_siteInfo.csv',
 'DFOSeaLice_Data.csv',
 'DFOSeaLice_Info.csv',
 'IndustrySeaLice_Data.csv',
 'IndustrySeaLice_Info.csv',
 'README.md']

#### Wild data

In [6]:
fish_data = pd.read_csv(f'{data_dir}BroughtonSeaLice_fishData.csv', encoding='ISO-8859-1')
site_data = pd.read_csv(f'{data_dir}BroughtonSeaLice_siteData.csv', encoding='ISO-8859-1')

/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
fish_info = pd.read_csv(f'{data_dir}BroughtonSeaLice_fishInfo.csv', encoding='ISO-8859-1')
site_info = pd.read_csv(f'{data_dir}BroughtonSeaLice_siteInfo.csv', encoding='ISO-8859-1')

#### Farm data

In [8]:
dfo_data = pd.read_csv(f'{data_dir}DFOSeaLice_Data.csv')
dfo_info = pd.read_csv(f'{data_dir}DFOSeaLice_Info.csv')

In [9]:
industry_data = pd.read_csv(f'{data_dir}IndustrySeaLice_Data.csv', encoding='ISO-8859-1', low_memory=False)
industry_info = pd.read_csv(f'{data_dir}IndustrySeaLice_Info.csv', encoding='ISO-8859-1', low_memory=False)

##### Data/Feature Engineering

Possible input scenarios
- 2001-2018: We have to trust that the model can work with the large amounts of NaN values in earlier years, both in wild data and no farmed data until 2011 
- 2003-2018: 2003 is the first year we have data starting in March


## Setting overall constants

Things to set here 
- Years to analyse
- Within-season date range
- Accepted ranges
- Resampling dates

In [10]:
analysis_years = list(range(2003, 2018))

In [11]:
analysis_months = list(range(3, 7))

In [12]:
dow_dict = {
    1: 'MON',
    2: 'TUE',
    3: 'WED',
    4: 'THU', 
    5: 'FRI', 
    6: 'SAT', 
    7: 'SUN'
}

def get_dow(dt_obj):
    dow_text = dt_obj.isoweekday()
    return(dow_dict[dow_text])

In [13]:
wild_locations = site_data['location'].unique()

## Setting up response

### Unified adult count
This is one possible response

In [14]:
adult = fish_data[['Lep_PAmale', 'Lep_PAfemale', 
                   'Lep_male', 'Lep_gravid',
                   'Lep_nongravid', 'unid_PA',
                   'unid_adult']].sum(axis=1)

In [15]:
fish_data_date = pd.to_datetime(fish_data[['year', 'day', 'month']])

In [16]:
response = pd.DataFrame({'count':adult.values, 
                         'location':fish_data['location'].values,
                         'datetime': fish_data_date})

In [17]:
response_glacier = response[response['location'] == 'Glacier']

In [18]:
year_df_list = []
for year in analysis_years:
    subset = response_glacier[response_glacier['datetime'].dt.year == year]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
    subset.sort_values('datetime', inplace=True)
    subset_resampled = subset.resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                       on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_df_list.append(subset_resampled)
Y_glacier = pd.concat(year_df_list)

/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [19]:
Y_glacier

,count
datetime,
2002-12-25,NaN
2003-01-01,NaN
2003-01-08,NaN
2003-01-15,NaN
2003-01-22,NaN
...,...
2017-11-26,0.0
2017-12-03,0.0
2017-12-10,0.0


resample by week for average

This is the code I was formerly using for the resampling, it's pretty inconsistent

Y_glacier = response_glacier.resample('W', on='datetime', label='left').apply(np.nanmean)

## Setting up inputs - wild data

## Setting up inputs - wild data

In [20]:
# non-motile lice
juvenile = pd.DataFrame(fish_data[['Lep_cope', 'chalA',
                      'chalB', 'Caligus_cope',
                      'unid_cope', 'chal_unid']].sum(axis=1)).rename({0: 'count'}, axis=1)
juvenile['datetime'] = fish_data_date
juvenile['location'] = fish_data['location']

In [21]:
year_juv_list = []
for year in analysis_years:
    subset = juvenile[juvenile['datetime'].dt.year == year]
    for loc in wild_locations:
        subset = subset.append({
            'datetime': datetime.datetime(year, 1 , 1),
            'location': loc,
            'count': np.nan
        }, ignore_index=True)
        subset = subset.append({
            'datetime': datetime.datetime(year, 12 , 31),
            'location': loc,
            'count': np.nan
        }, ignore_index=True)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_juv_list.append(subset_resample)
X_wild_juv = pd.concat(year_juv_list)

This is the code I was previously using to resample the juvenile counts, now outdated

X_wild_juv = juvenile.groupby('location').resample('W', on='datetime', label='left').apply(np.nanmean)

Reducing dimensionality - averaging is probably okay

In [22]:
site_data['datetime'] = pd.to_datetime(site_data[['year', 'month', 'day']])

In [23]:
year_temp_list = []
for year in analysis_years:
    subset = site_data.loc[(site_data['datetime'].dt.year == year), ['datetime', 'temp', 'location']]
    for loc in wild_locations:
        subset = subset.append({
            'datetime': datetime.datetime(year, 1 , 1),
            'location': loc,
            'temp': np.nan
        }, ignore_index=True)
        subset = subset.append({
            'datetime': datetime.datetime(year, 12 , 31),
            'location': loc,
            'temp': np.nan
        }, ignore_index=True)
    subset.sort_values('datetime', inplace=True)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_temp_list.append(subset_resample)
X_wild_temp = pd.concat(year_temp_list)

This is how I used to get these inputs, now outdated

X_wild_temp = site_data[['datetime', 'temp', 'location']]
X_wild_temp = X_wild_temp.groupby('location').resample('w', on='datetime', label='left').apply(np.nanmean)

In [24]:
year_sal_list = []
for year in analysis_years:
    subset = site_data.loc[(site_data['datetime'].dt.year == year), ['datetime', 'salt', 'location']]
    for loc in wild_locations:
        subset = subset.append({
            'datetime': datetime.datetime(year, 1 , 1),
            'location': loc,
            'salt': np.nan
        }, ignore_index=True)
        subset = subset.append({
            'datetime': datetime.datetime(year, 12 , 31),
            'location': loc,
            'salt': np.nan
        }, ignore_index=True)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(method='linear')
    year_sal_list.append(subset_resample)
X_wild_sal = pd.concat(year_sal_list)

How I used to get these inputs, now outdated

X_wild_sal = site_data[['datetime', 'salt', 'location']]
X_wild_sal = X_wild_sal.groupby('location').resample('w', on='datetime', label='left').apply(np.nanmean)

## Setting up inputs - farm data

- There are of course concerns with using the industry counts as they *may* be of very low quality (I need to look much more extensively into this data)
    - Should look into what kind of predictive differences including/excluding these has 

Inputs that we want

44 - Sargeunts Pass

41 - Doctor Islet

45 - Humphrey Rock

56 - Glacier Falls

54 - Simoom Sound

50 - Burdwood Islands

53 - Sir Edmond Bay

49 - Wicklow Point 

In [25]:
industry_data[industry_data['Site Common Name'].str.contains('Sir')]['Site Common Name'].unique()
# Can't find Simoom Sound??

array(['Sir Edmund Bay'], dtype=object)

To reduce dimensionality:

Sum farm data, implementing # of fish for some sort of area infection indicator

In [26]:
relevant_farms = ['Sargeaunt Pass',
                  'Doctor Islets',
                  'Humphrey Rock',
                  'Simoom Sound*',
                  'Burdwood',
                  'Glacier Falls',
                  'Sir Edmund Bay',
                  'Wicklow Point'
                 ]
relevant_farms_iterable = ['Sargeaunt Pass',
                           'Doctor Islets',
                           'Humphrey Rock',
                           'Burdwood',
                           'Glacier Falls',
                           'Sir Edmund Bay',
                           'Wicklow Point'
                          ]


In [27]:
industry_data[industry_data['Site Common Name'].str.contains('|'.join(relevant_farms))]['Site Common Name'].unique()

array(['Burdwood', 'Sir Edmund Bay', 'Doctor Islets', 'Glacier Falls',
       'Wicklow Point', 'Sargeaunt Pass', 'Humphrey Rock',
       'Wicklow Point - post treatment', 'Wicklow Point - pre treatment',
       'Wicklow Point - post-treatment', 'Wicklow Point - pre-treatment'],
      dtype=object)

From what I can tell - most of these farms don't actually have a treatment indicator

Wondering if this is significant? 

I also can't find Simoom Sound, maybe it's here under a different name? 

In [28]:
comments_examples = industry_data[industry_data['Site Common Name'] == 'Burdwood']['Comments'].unique()

This is how I was resampling the industry data before

X_industry = relevant_farm_data[['datetime',
                            'Site Common Name',
                            'Average L. salmonis motiles per fish',
                            'Average chalimus per fish']]
X_industry = X_industry.groupby('Site Common Name').resample('W', on='datetime', label='left').mean()

The "Recent failure to control sea louse..." paper should have all the data I need until 2015

In [29]:
relevant_farm_data = industry_data[industry_data['Site Common Name'].str.contains('|'.join(relevant_farms))]

relevant_farm_data['Day'] = 1
month_map = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
relevant_farm_data['month'] = relevant_farm_data['Month'].map(month_map)
relevant_farm_data['datetime'] = pd.to_datetime(relevant_farm_data[['Year', 'month', 'Day']])

relevant_farm_data = relevant_farm_data[relevant_farm_data['datetime'].dt.year.isin(analysis_years)]

year_industry_list = []
for year in analysis_years:
    subset = relevant_farm_data.loc[(relevant_farm_data['datetime'].dt.year == year), 
                                   ['datetime', 'Site Common Name', 'Average L. salmonis motiles per fish']]
    
    for i, farm in enumerate(relevant_farms_iterable):
        subset = subset.append({
            'datetime': datetime.datetime(year, 1 , 1),
            'Site Common Name': farm,
            'Average L. salmonis motiles per fish': np.nan
        }, ignore_index=True)
        subset = subset.append({
            'datetime': datetime.datetime(year, 12 , 31),
            'Site Common Name': farm,
            'Average L. salmonis motiles per fish': np.nan
        }, ignore_index=True)
            
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('Site Common Name').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                                 on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_industry_list.append(subset_resample)
X_industry = pd.concat(year_industry_list)


/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

### Forming array for model input

In [39]:
np.nanmean(juv_sub, axis=1)

(17,)

In [55]:
arrays_to_stack = []

for year in analysis_years:
    juv_sub = np.nanmean(X_wild_juv[(X_wild_juv.index.get_level_values(1).year == year) & 
                         X_wild_juv.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values, axis=1)
    temp_sub = np.nanmean(X_wild_temp[(X_wild_temp.index.get_level_values(1).year == year) & 
                           X_wild_temp.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values, axis=1)
    sal_sub = np.nanmean(X_wild_sal[(X_wild_sal.index.get_level_values(1).year == year) & 
                         X_wild_sal.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values, axis=1)
    ind_sub = np.nanmean(X_industry[(X_industry.index.get_level_values(1).year == year) & 
                         X_industry.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values, axis=1)
    
    year_array = np.column_stack((juv_sub, temp_sub, sal_sub, ind_sub))
    arrays_to_stack.append(year_array)
    
test_X = np.stack(arrays_to_stack, axis=0)

/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


In [58]:
test_X.shape

(15, 17, 4)

In [59]:
arrays_to_stack = []

for year in analysis_years:
    year_Y = Y_glacier[(Y_glacier.index.year == year) & 
                       Y_glacier.index.month.isin(analysis_months)].values
#     if year_Y.shape[0] == 21:
#         year_Y = np.append(year_Y, np.nan)
#         year_Y = year_Y.reshape(22, 1)
    arrays_to_stack.append(year_Y)
    
test_Y = np.stack(arrays_to_stack, axis=0)
test_Y = test_Y

In [60]:
test_Y.shape

(15, 17, 1)

# Experimenting with models

## Import packages and set up functions

Write normalising functions here (let's go with MinMaxScaling for future implementations)

## Setup Data

### Normalise and fill NAs

This section is pretty bad!

This is an issue being worked on, I need to:
* Render the data stationary
* Use a different scaling function
* As a QOL update - right now these cells matter heavily on the proper order of cells being executed (top to bottom) in the notebook, that's pretty bad and should be fixed

In [63]:
X_mean = np.nanmean(test_X, axis=1)
for i in range(0, X_mean.shape[0]):
    test_X[[i]] = test_X[[i]] - X_mean[[i]]
X_std = np.nanstd(test_X, axis=1)
for i in range(0, X_std.shape[0]):
    test_X[[i]] = test_X[[i]] - X_std[[i]]
test_X = np.nan_to_num(test_X)

/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/EEB498/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [64]:
Y_mean = np.nanmean(test_Y, axis=1)
for i in range(0, Y_mean.shape[0]):
    test_Y[[i]] = test_Y[[i]] - Y_mean[[i]]
Y_std = np.nanstd(test_Y, axis=1)
for i in range(0, Y_std.shape[0]):
    test_Y[[i]] = test_Y[[i]] - Y_std[[i]]
test_Y = np.nan_to_num(test_Y)

### Split into train and test

In [65]:
train_X = test_X[:-3]
train_Y = test_Y[:-3]

eval_X = test_X[-3:]
eval_Y = test_Y[-3:]

## Baseline

### Overall Avg

### Naive seasonal

In [38]:
def naive_seasonal(year: int, reference_data: pd.DataFrame):
    '''
    Naive seasonal model
    This model takes a year to be predicted in and returns the last known year's values
    
    year: Year to be predicted
    reference_data: Pandas dataframe of the test/input Y data, must have a DatetimeIndex index
    '''
    
    
    pred_subset = reference_data[reference_data.index.year == year]
    preds = pred_subset[pred_subset.index.month.isin(analysis_months)]
    
    return(np.array(preds))

### ARIMA

## Non-parametric

Some high level notes
- This analysis is very rough and is in no way final!!!
- The normalising, feature engineering, etc. is probably the roughest part of all of this. I don't expect it to have an effect on model choice but by no means should the input/output data be taken verbatim as what I intend to use
- Optimizers: some reading has shown that RMSprop is the suggested optimiser for RNNs, this also coincides with François Chollet's use of optimisers so for now I am going with this one for RNNs and ADAM as the default for all others. This requires further research!
- Reference points: 
     - ARIMA
     - Mean & SD

### Setting up tensorboard logs

### LSTM

Long short-term memory (LSTM)

This model is the most advanced RNN for sequential data, so the highest potential upside in gaining value from features but may also be overkill so make sure to compare it to some other RNN baselines

In [61]:
!rm -rf ./logs/ 
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [66]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(100, input_shape=(17, 4),
               return_sequences=True,
               activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='rmsprop', 
              loss='mse',
              metrics=['acc'])

history = model.fit(train_X, train_Y, 
                    epochs=20, batch_size=16,
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])

Train on 9 samples, validate on 3 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
9/9 [==============================] - 1s 82ms/sample - loss: 0.7108 - acc: 0.1307 - val_loss: 0.0463 - val_acc: 0.2353
Epoch 2/20
9/9 [==============================] - 0s 2ms/sample - loss: 0.4744 - acc: 0.1307 - val_loss: 0.0171 - val_acc: 0.2353
Epoch 3/20
9/9 [==============================] - 0s 2ms/sample - loss: 0.3560 - acc: 0.1307 - val_loss: 0.0189 - val_acc: 0.2353
Epoch 4/20
9/9 [==============================] - 0s 3ms/sample - loss: 0.2941 - acc: 0.1307 - val_loss: 0.0237 - val_acc: 0.2353
Epoch 5/20
9/9 [==============================] - 0s 3ms/sample - loss: 0.3043 - acc: 0.1307 - val_loss: 0.0352 - val_acc: 0.2353
Epoch 6/20
9/9 [==============================] - 0s 2ms/sample - loss: 0.2668 - acc: 0.1307 - val_loss: 0.0287 - val_acc: 0.2353
Epoch 7/20
9/9 [==============================] - 0s 3ms/sample - loss: 0.2224 - acc: 0.1307 - val_loss: 0.0301 - val_acc: 0.2353

In [67]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
train_predictions = model.predict(train_X)

# Train RMSE
metrics.mean_squared_error(train_Y.reshape(train_Y.shape[0], train_Y.shape[1] * train_Y.shape[2]),
                           train_predictions.reshape(train_predictions.shape[0], train_predictions.shape[1] * train_predictions.shape[2]))

In [ ]:
test_predictions = model.predict(eval_X)

# Test RMSE
metrics.mean_squared_error(eval_Y.reshape(eval_Y.shape[0], eval_Y.shape[1] * eval_Y.shape[2]),
                           test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * train_predictions.shape[2]))

### CNN

1D Convnet

1D convnets have sometimes shown to be more efficient with data than similarly sized RNNs when dealing with small problems. Maybe that means they're a good application here? Each output timestep takes power from its neighbours, so this is assuming some sort of autoregression.

In [ ]:
from keras.models import Sequential
from keras import layers

In [44]:
train_Y.shape

(12, 17, 1)

In [45]:
model = Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=2,
                        activation='relu',
                        input_shape=(22,6)))
model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1))
# 1D Convnets cannot return sequences, they converge to one value...
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['acc'])

history = model.fit(train_X, train_Y.reshape(12, 22), 
                    epochs=100, batch_size=16,
                    validation_split=0.2)

ValueError: cannot reshape array of size 204 into shape (12,22)

## Parametric

### ARIMA